In [2]:
#import libraries and dependencies
import pandas as pd
import yfinance as yf
import numpy as np
from scipy.stats import zscore
import matplotlib.pyplot as plt
from scipy import stats
import datetime as dt
from sklearn import linear_model
from pandas import Timestamp
from scipy.stats import linregress

pd.set_option('display.max_columns', None)


In [3]:
# Step 1: Choose an ETF and identify assets
df_spy=pd.read_excel('https://www.ssga.com/us/en/institutional/etfs/library-content/products/fund-data/etfs/us/holdings-daily-us-en-spy.xlsx',header=4)
etf_ticker=df_spy['Ticker'].dropna().to_list() #identifying the assets inside the s&p500

# Step 2: Retrieve historical data for the ETF
historical_data = yf.download(etf_ticker, period='1y') #looks good


[*********************100%%**********************]  504 of 504 completed


3 Failed downloads:
['BRK.B', '-']: Exception('%ticker%: No data found, symbol may be delisted')
['BF.B']: Exception('%ticker%: No price data found, symbol may be delisted (period=1y)')


In [4]:
# Step 3: Calculate the price momentum factors
# Here, we use a simple 1-month rate of change

#Calculating the 1st factor (52 week trend line)
HIST_DATA=historical_data[historical_data.columns].pct_change(periods=20).dropna(how='all',axis=0) #George's Idea
HIST_DATA=HIST_DATA.drop(HIST_DATA.columns[0],axis=1)['Adj Close'] #first column is all NAN's

#Trend line (slope) for 1 asset "AAL"  credit: https://stackoverflow.com/questions/53100393/how-to-get-slope-from-timeseries-data-in-pandas
# https://www.appsloveworld.com/pandas/100/13/how-to-get-slope-from-timeseries-data-in-pandas?expand_article=1    
# Extract numeric representation of dates for linear regression
def getslope(data):
    x_values = np.arange(len(data.index.values))
    
    # Perform linear regression
    slope, intercept, r_value, p_value, std_err = linregress(x_values, data)
    
    return slope#Thanks to ChatGPT for helping me get the slope as i did not know how to do it for a time series data

#now lets get the slope for all the assets
#getslope(HIST_DATA['AAL']) test case

slopes=HIST_DATA.apply(getslope) #it's rounding but that's fine
#HIST_DATA['AAL'].plot(kind='line') #plot of 1 trend line 


#Step 4:
slopes=slopes.dropna() #the nans need to be removed
assets_Zscores=zscore(slopes) #get the Z score
#assets_Zscores Always nice to take a quick look

#Step 5: Identify baskets (I Think it works)
#resource: https://www.freshbooks.com/glossary/financial/z-score#:~:text=What%20Is%20a%20Good%20Z,considered%20above%20the%20standard%20score.
#I'll define my boundary at 3 and even though i hate shorting stocks but for this project we will short those under 1.8
#link: https://www.investopedia.com/terms/a/altman.asp#:~:text=A%20score%20below%201.8%20signals,value%20is%20closer%20to%201.8.
buy=assets_Zscores[assets_Zscores>1.8]
short=assets_Zscores[assets_Zscores<1.8]

C:\Users\georg\AppData\Local\Temp\ipykernel_29656\1736214376.py:5: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Call ffill before calling pct_change to retain current behavior and silence this warning.
  HIST_DATA=historical_data[historical_data.columns].pct_change(periods=20).dropna(how='all',axis=0) #George's Idea


In [5]:
#Step 6: Back test 
#To run this for 5 years we can get the data for all of our assets for the last 5 years and for each of the 60 months buy the suggested stock
#to see it's returns for those  periods

historical_data = yf.download(etf_ticker, start='2018-01-01', end='2023-01-01')
historical_data=historical_data['Adj Close'].drop(columns='-')

#Calculating the 1st factor (52 week trend line)
HIST_DATA=historical_data[historical_data.columns].pct_change(periods=20).dropna(how='all',axis=0)
assets_Zscores=zscore(HIST_DATA.apply(getslope).dropna()) #get the Z score
buy=assets_Zscores[assets_Zscores>1.8]
short=assets_Zscores[assets_Zscores<1.8]
buynow=buy[0:15] #lets buy this on january first and see how much we would have made
shortnow=short[0:10] #We can short these

[*********************100%%**********************]  504 of 504 completed


5 Failed downloads:
['BRK.B', '-']: Exception('%ticker%: No timezone found, symbol may be delisted')
['KVUE', 'VLTO']: Exception("%ticker%: Data doesn't exist for startDate = 1514782800, endDate = 1672549200")
['BF.B']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2018-01-01 -> 2023-01-01)')


C:\Users\georg\AppData\Local\Temp\ipykernel_29656\2195601254.py:9: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Call ffill before calling pct_change to retain current behavior and silence this warning.
  HIST_DATA=historical_data[historical_data.columns].pct_change(periods=20).dropna(how='all',axis=0)


In [6]:
print(buynow,shortnow)

ALB     2.021040
APA     2.830595
BBWI    1.921529
DVN     3.344415
EOG     2.248615
EQT     3.026700
FANG    2.396167
FCX     1.976041
FSLR    2.541589
HAL     3.081373
KHC     1.891219
MPC     2.017218
MRO     2.817096
NUE     2.062247
OXY     2.998400
dtype: float64 A       0.364925
AAL     0.884792
AAPL   -0.692455
ABBV    1.191747
ABT    -0.766205
ACGL    0.895966
ACN    -0.308543
ADBE   -1.696200
ADI    -0.028365
ADM     0.875900
dtype: float64


In [ ]:
#STEPS 6 & 7 NO IDEA WHAT TO DO NOW